### Author: Shams

**Description:**
This is where I build the tokenizer from the ground up. I am not loading a pre-trained one. I am taking the raw BPE algorithm, adding the BART-specific rules (like using 'Metaspace' and the specific start/end tags), and training it on my cleaned data. This way it learns Egyptian Arabic correctly.

**The making:**
Instead of loading a ready-made tokenizer, I initialize a raw `BPE` model. I manually set the pre-tokenizer to `Metaspace` (which handles spaces as underscores, crucial for BART/RoBERTa). Then I define the Trainer with my vocabulary size (90k). Finally, and most importantly, I attach a `TemplateProcessing` post-processor that forces every sentence to look like `<s> sentence </s>`. This matches exactly what the BART model expects.

### 1. Setup and Imports
Installing the libraries and importing what I need. I use `tokenizers` for the heavy lifting and `transformers` to wrap the result.

In [ ]:
!pip install tokenizers datasets transformers -q
print("Libraries installed.")

import os
import shutil
import time
from datasets import load_from_disk
from tokenizers import (
    Tokenizer,
    models,
    pre_tokenizers,
    trainers,
    processors,
    decoders
)
from transformers import PreTrainedTokenizerFast
import json

### 2. Configuration
Here I define the rules. I'm using a vocabulary of 90,000 words because Arabic is rich. I also define the special tokens exactly like the original BART model uses (`<s>`, `</s>`, etc.) so they are compatible.

In [ ]:
class TokenizerConfig:
    # simple flag to run a fast test if needed
    IS_TEST_RUN = False
    
    # where the cleaned data lives
    INPUT_DATA_PATH = "/kaggle/input/spre-tokenization/MyPreTokenizedData"
    
    # where to put the output
    SAVE_DIR = "/kaggle/working/ARZ-EN-BART-Tokenizer/"
    
    TEXT_COLUMN = 'pretokenized'

    # 90k is a good sweet spot for bilingual (ar/en) data
    VOCAB_SIZE = 90000
    MIN_FREQUENCY = 7
    
    # I split data into chunks to save RAM
    ROWS_PER_CHUNK_FILE = 4_000_000
    TEXT_CHUNK_DIR = "/kaggle/working/training_text_chunks/"

    # BART uses these specific symbols. I need to match them.
    UNK_TOKEN = "<unk>"
    PAD_TOKEN = "<pad>"
    MASK_TOKEN = "<mask>"
    CLS_TOKEN = "<s>"    # Start of sentence
    EOS_TOKEN = "</s>"   # End of sentence
    
    SPECIAL_TOKENS = [
        UNK_TOKEN, PAD_TOKEN, MASK_TOKEN, CLS_TOKEN, EOS_TOKEN
    ]

print("Config loaded.")

### 3. File Streaming
I can't load the whole dataset into RAM at once. This function reads the dataset row by row and writes it into smaller text files (`chunks`). The tokenizer will read these files one by one later.

In [ ]:
def get_training_files(config: TokenizerConfig) -> list[str]:
    print(f"\nStreaming dataset to text files...")
    print(f"  -> Loading from: {config.INPUT_DATA_PATH}")
    try:
        main_dataset = load_from_disk(config.INPUT_DATA_PATH)
        print(f"  -> Loaded. Rows: {len(main_dataset)}")
    except Exception as e:
        print(f"  -> Failed to load data. Error: {e}")
        return []

    os.makedirs(config.TEXT_CHUNK_DIR, exist_ok=True)
    file_paths = []
    file_number = 1

    print(f"  -> Writing chunks to: {config.TEXT_CHUNK_DIR}")
    
    f = open(os.path.join(config.TEXT_CHUNK_DIR, f"chunk_{file_number}.txt"), "w", encoding="utf-8")
    
    for i, record in enumerate(main_dataset):
        # stop early if this is just a test
        if config.IS_TEST_RUN and i > 10_000_000:
            print(f"    -> Test mode: Stopping early.")
            break
            
        # close current file and start a new one if it gets too big
        if i > 0 and i % config.ROWS_PER_CHUNK_FILE == 0:
            f.close()
            file_paths.append(f.name)
            print(f"    - Saved chunk: {os.path.basename(f.name)}")
            file_number += 1
            f = open(os.path.join(config.TEXT_CHUNK_DIR, f"chunk_{file_number}.txt"), "w", encoding="utf-8")
        
        # write the actual text to the file
        text_list = record.get(config.TEXT_COLUMN)
        if text_list and isinstance(text_list, list):
            f.write(" ".join(text_list) + "\n")

    f.close()
    file_paths.append(f.name)
    print(f"    - Saved final chunk: {os.path.basename(f.name)}")
    print(f"  -> Chunking done. Created {len(file_paths)} files.")
    return file_paths

### 4. Training Logic
This is the core logic. 
1. I start a blank BPE tokenizer.
2. I set `Metaspace` (this replaces spaces with a special underscore char, needed for RoBERTa/BART).
3. I train it on the files we made above.
4. **Crucial**: I attach a `post_processor`. This ensures every sentence automatically gets wrapped in `<s>` and `</s>` tags.

In [ ]:
def train_and_save_tokenizer(config: TokenizerConfig, file_paths: list[str]):
    print(f"\nTraining Tokenizer...")
    
    # starting from scratch with BPE
    backend_tokenizer = Tokenizer(models.BPE(unk_token=config.UNK_TOKEN))
    
    # bart/roberta need this specific pre-tokenizer to handle spaces correctly
    backend_tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()
    backend_tokenizer.decoder = decoders.Metaspace()

    # setting up the training rules
    trainer = trainers.BpeTrainer(
        vocab_size=config.VOCAB_SIZE,
        min_frequency=config.MIN_FREQUENCY,
        special_tokens=config.SPECIAL_TOKENS
    )

    print(f"  -> Training on {len(file_paths)} files...")
    start_time = time.perf_counter()
    backend_tokenizer.train(files=file_paths, trainer=trainer)
    end_time = time.perf_counter()
    print(f"  -> Training finished in {end_time - start_time:.2f}s")

    # finding the IDs for start and end tokens
    cls_token_id = backend_tokenizer.token_to_id(config.CLS_TOKEN)
    eos_token_id = backend_tokenizer.token_to_id(config.EOS_TOKEN)

    if cls_token_id is None or eos_token_id is None:
        raise RuntimeError(f"Something went wrong. Special tokens missing.")
        
    print(f"  -> Adding BART post-processing rules...")

    # this template tells the tokenizer: "put <s> at start and </s> at end"
    backend_tokenizer.post_processor = processors.TemplateProcessing(
        single=f"{config.CLS_TOKEN} $A {config.EOS_TOKEN}",
        pair=f"{config.CLS_TOKEN} $A {config.EOS_TOKEN} {config.EOS_TOKEN} $B {config.EOS_TOKEN}",
        special_tokens=[
            (config.CLS_TOKEN, cls_token_id),
            (config.EOS_TOKEN, eos_token_id)
        ],
    )

    print(f"  -> Wrapping in Hugging Face format...")
    
    final_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=backend_tokenizer,
        unk_token=config.UNK_TOKEN,
        pad_token=config.PAD_TOKEN,
        mask_token=config.MASK_TOKEN,
        cls_token=config.CLS_TOKEN,
        eos_token=config.EOS_TOKEN,
        sep_token=config.EOS_TOKEN,      # bart uses eos as separator
        bos_token=config.CLS_TOKEN,      # bart uses cls as beginning
        add_prefix_space=True            # essential for bart
    )

    print(f"  -> Vocab size: {len(final_tokenizer)}")

    os.makedirs(config.SAVE_DIR, exist_ok=True)
    print(f"  -> Saving to: {config.SAVE_DIR}")
    
    # saving the raw backend JSON
    backend_tokenizer.save(os.path.join(config.SAVE_DIR, "tokenizer.json"))
    
    # saving the friendly HF config files
    final_tokenizer.save_pretrained(config.SAVE_DIR)
    
    print(f"  -> Saved. Ready for verification.")

### 5. Main Execution
Running the whole pipeline: config, chunking, training, and cleanup.

In [ ]:
def main():
    print("===== BART Tokenizer Training =====")
    total_start_time = time.perf_counter()
    
    if config.IS_TEST_RUN:
        print("  -> Mode: Test Run")
    else:
        print("  -> Mode: Production Run")
    print(f"  -> Output: {config.SAVE_DIR}")

    try:
        # step 1: create text files
        training_files = get_training_files(config)
        
        if not training_files:
            raise RuntimeError("No files created.")

        # step 2: train
        train_and_save_tokenizer(config, training_files)

    except Exception as e:
        print(f"\n--- CRASHED ---")
        print(f"  -> {e}")
    
    finally:
        # step 3: cleanup
        print(f"\n--- Cleanup ---")
        if os.path.exists(config.TEXT_CHUNK_DIR):
            shutil.rmtree(config.TEXT_CHUNK_DIR)
            print(f"  -> Removed temp files.")
        else:
            print(f"  -> Nothing to clean.")

    total_end_time = time.perf_counter()
    print(f"\n===== Finished in {total_end_time - total_start_time:.2f}s =====")


# create config and run
config = TokenizerConfig()
if __name__ == "__main__":
    main()

### 6. Verification Tests
I don't blindly trust code. I need to load the saved tokenizer and run tests.
1. **Load Test**: Does it open?
2. **Special Tokens**: Are `<s>` and `</s>` mapped correctly?
3. **Round-Trip**: Can I encode a weird sentence and get the exact same string back?
4. **Pair Test**: If I give it two sentences, does it put the separators `</s> </s>` in the middle?

In [ ]:
import os
from transformers import PreTrainedTokenizerFast
import json

# --- CHECK CONFIG ---
try:
    TOKENIZER_PATH = config.SAVE_DIR
    print(f"  -> Using path from memory: {TOKENIZER_PATH}")
except NameError:
    TOKENIZER_PATH = "/kaggle/working/ARZ-EN-BART-Tokenizer/"
    print(f"  -> Using default path: {TOKENIZER_PATH}")

if not os.path.exists(TOKENIZER_PATH):
    print(f"Directory not found. Run training first.")
else:
    try:
        # --- 1. LOAD TEST ---
        print("\n--- [1] Loading ---")
        tokenizer = PreTrainedTokenizerFast.from_pretrained(TOKENIZER_PATH)
        print(f"  -> Loaded. Vocab size: {tokenizer.vocab_size}")

        # --- 2. SPECIAL TOKENS TEST ---
        print("\n--- [2] Checking Special Tokens ---")
        assert tokenizer.unk_token == "<unk>"
        assert tokenizer.pad_token == "<pad>"
        assert tokenizer.mask_token == "<mask>"
        assert tokenizer.cls_token == "<s>"
        assert tokenizer.eos_token == "</s>"
        # check aliases
        assert tokenizer.bos_token == tokenizer.cls_token
        assert tokenizer.sep_token == tokenizer.eos_token
        print("  -> Tokens match BART standard.")
        print(f"  -> IDs: CLS={tokenizer.cls_token_id}, EOS={tokenizer.eos_token_id}")

        # --- 3. ROUND TRIP TEST ---
        print("\n--- [3] Round-Trip Tests ---")
        
        complex_sentences = [
            "المفروض يعني محدش يتناك على التاني لحد مانلاقي حل في الموضوع ابن العرص دا 12345.",
            "The tokenizer's interoperability, tested circa 2024-2025, must be 100% perfect.",
            "!@#$%^&*_)(+1548/*@يىتىيسالةيكنةبىسنمf dkjn vjkfnvjdsvsnfvjfbvjbfvhdsvkdsvnfvo   ufnf",
            "I told him to 'stop' 5 times, بس هو مصمم يعك الدنيا.",
            "اللامركزية في البلوكتشين بتدينا شفافية ومحدش يقدر يتحكم في الداتا لوحده."
        ]
        
        for i, text in enumerate(complex_sentences):
            # encode then decode
            encoded_ids = tokenizer.encode(text)
            decoded = tokenizer.decode(encoded_ids, skip_special_tokens=True)
            
            # verify they are identical
            if decoded != text:
                 print(f"Mismatch in test {i+1}!")
                 print(f"Orig: {text}")
                 print(f"Decoded: {decoded}")
                 raise AssertionError("Round-trip failed.")
            print(f"  -> Test {i+1} passed.")

        print("  -> All text tests passed.")


        # --- 4. TEMPLATE TEST ---
        print("\n--- [4] Template Test ---")
        text_pair_1 = "First part."
        text_pair_2 = "الجزء التاني."
        encoded_pair = tokenizer(text_pair_1, text_pair_2)
        
        ids = encoded_pair['input_ids']
        eos_id = tokenizer.eos_token_id
        cls_id = tokenizer.cls_token_id
        
        # look for double EOS in the middle
        double_eos_index = -1
        for i in range(len(ids) - 1):
            if ids[i] == eos_id and ids[i+1] == eos_id:
                double_eos_index = i
                break
                
        assert ids[0] == cls_id, "Start token missing"
        assert ids[-1] == eos_id, "End token missing"
        assert double_eos_index != -1, "Separator (</s> </s>) missing"
        print(f"  -> Found separator at index {double_eos_index}.")

        print("\n===== VERIFIED =====")

    except Exception as e:
        print(f"\n--- TEST FAILED ---")
        print(e)

### 7. Internal Inspection
Just to be absolutely sure, I'm opening the JSON files themselves. I need to verify that `add_prefix_space` is `true` inside `tokenizer.json`, otherwise, the model will fail silently during inference.

In [ ]:
import os
import json

print("===== Internal JSON Check =====")

# get path
try:
    TOKENIZER_PATH = config.SAVE_DIR
except NameError:
    TOKENIZER_PATH = "/kaggle/working/ARZ-EN-BART-Tokenizer/"

# 1. Check tokenizer_config.json
config_path = os.path.join(TOKENIZER_PATH, "tokenizer_config.json")
try:
    print("--- tokenizer_config.json ---")
    with open(config_path, 'r') as f:
        config_data = json.load(f)
    
    if config_data.get("add_prefix_space") is True:
        print("  -> add_prefix_space is True (Good).")
    else:
        print("  -> WARNING: add_prefix_space is NOT True.")
except Exception as e:
    print(f"Error reading config: {e}")


# 2. Check tokenizer.json internals
core_path = os.path.join(TOKENIZER_PATH, "tokenizer.json")
try:
    print("\n--- tokenizer.json ---")
    with open(core_path, 'r') as f:
        core_data = json.load(f)

    # check pre-tokenizer type
    pre_tok_type = core_data.get('pre_tokenizer', {}).get('type')
    if pre_tok_type == "Metaspace":
         print("  -> Pre-tokenizer is Metaspace (Good).")
    else:
         print(f"  -> WARNING: Pre-tokenizer is {pre_tok_type}.")

    # check post-processor logic for the pairs
    # we expect the pair template to include special tokens for A and B
    post_processor = core_data.get('post_processor', {})
    try:
        pair_template = post_processor.get('pair', [])
        # just checking if we have enough items in the template list to constitute a full structure
        if len(pair_template) >= 5:
             print("  -> Post-processor template looks correct.")
        else:
             print("  -> WARNING: Post-processor template looks too short.")
    except Exception:
        print("  -> Error checking post-processor.")

except Exception as e:
    print(f"Error reading core file: {e}")

print("\nDone checking internals.")